In [1]:
import pandas as pd
import tarfile 
import os

In [2]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
import numpy as np
from keras.models import Model
from keras.layers import Input, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

from nltk.stem.snowball import SnowballStemmer

import re

import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [3]:
data_path = os.getcwd()

In [4]:
classes = os.listdir(data_path + '\\20_newsgroups\\20_newsgroups')

In [5]:
classes

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [6]:
labels_index = {}
labels = []
texts = []
for names in classes:
    path = os.path.join(data_path + '\\20_newsgroups\\20_newsgroups',names)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[names] = label_id
        for name in os.listdir(path):
                fpath = os.path.join(path, name)
                f = open(fpath, encoding='latin-1')
                t = f.read()
                i = t.find('\n\n')
                if i>0:
                    t=t[i:]
                texts.append(t)
                f.close
                labels.append(label_id)
         #print('No. of files in {:} = {:}'. format(names,len(labels)))
print(labels_index) 

{'alt.atheism': 0, 'comp.graphics': 1, 'comp.os.ms-windows.misc': 2, 'comp.sys.ibm.pc.hardware': 3, 'comp.sys.mac.hardware': 4, 'comp.windows.x': 5, 'misc.forsale': 6, 'rec.autos': 7, 'rec.motorcycles': 8, 'rec.sport.baseball': 9, 'rec.sport.hockey': 10, 'sci.crypt': 11, 'sci.electronics': 12, 'sci.med': 13, 'sci.space': 14, 'soc.religion.christian': 15, 'talk.politics.guns': 16, 'talk.politics.mideast': 17, 'talk.politics.misc': 18, 'talk.religion.misc': 19}


In [8]:
contractions = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [9]:
def expand_contractions(text):
    for word in text.split():
        if word.lower() in contractions:
            text = text.replace(word, contractions[word.lower()])
    return text

In [10]:
texts = [expand_contractions(re.sub('’', "'", text)) for text in texts]

In [11]:
texts = [text.lower() for text in texts]

In [12]:
def scrub_words(text):
    text = re.sub('[$|@](\w)*[\s]',' ',text) ## removing tagged words
    text = re.sub('\n',' ',text) ### removing new line character
    text = re.sub('(?P<url>https?://[^\s]+)',' ',text) ### removing http links
    text = re.sub("(\\W|\\d)",' ',text) ### removing non alphanumeric and digits
    text = re.sub(r"\b[a-zA-Z]\b", "", text) ### removing single characters
    text = re.sub("\s+", ' ', text) ### removing extra white spaces
    
    return text

In [13]:
texts = [scrub_words(text) for text in texts]

In [14]:
#snowball=SnowballStemmer('english')
stopword_list = stopwords.words('english')

## stopword removal

In [15]:
def remove_stopwords(text):
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in stopword_list]
    return ' '.join(tokens)

In [16]:
texts = [remove_stopwords(text) for text in texts]

In [17]:
from nltk import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [18]:
def lemma(text):
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

In [19]:
texts = [lemma(text) for text in texts]

In [20]:
texts[0][0:100]

'archive name atheism resource alt atheism archive name resource last modified december version athei'

## model building

In [124]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
word_Index = tokenizer.word_index
vocab_Size = len(word_Index) + 1
print('Found %s unique tokens.' % vocab_Size)

Found 104728 unique tokens.


In [125]:
X = tokenizer.texts_to_sequences(texts)

In [127]:
X = pad_sequences(X, maxlen=1000)

In [128]:
temp_array = np.array(labels)
len(temp_array[temp_array==0])

1001

In [ ]:
labels = to_categorical(np.array(labels))

In [131]:
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
training_set = int(0.8*len(indices))
X = X[indices]
labels = labels[indices]
x_train = X[:training_set]
y_train = labels[:training_set]
x_test = X[training_set:]
y_test = labels[training_set:]

In [133]:
MAX_SEQUENCE_LENGTH = 1000
Embedding_dim = 50
embedding_layer = Embedding(vocab_Size,Embedding_dim,input_length=MAX_SEQUENCE_LENGTH)

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(64, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(4)(x)
x = Conv1D(64, 5, activation='relu')(x)
x = MaxPooling1D(4)(x)
x = Conv1D(64, 5, activation='relu')(x)
x = MaxPooling1D(4)(x)  # global max pooling
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)

In [134]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 1000, 50)          5236400   
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 996, 64)           16064     
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 249, 64)           0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 245, 64)           20544     
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 61, 64)            0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 57, 64)            2054

In [135]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [136]:
model.fit(x_train, y_train, epochs=2)

Epoch 1/2
15998/15998 [==============================] - 144s 9ms/step - loss: 2.5476 - accuracy: 0.1396
Epoch 2/2
15998/15998 [==============================] - 133s 8ms/step - loss: 2.0545 - accuracy: 0.2704


In [137]:
glove_path = r'C:\Users\AMAN\Downloads\20_newsgroups'
embedding_index={}
f = open(os.path.join(glove_path,'glove.6B.50d.txt'), encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coef = np.asarray(values[1:], dtype='float32')
    embedding_index[word] = coef
f.close()

In [138]:
embedding_Matrix = np.zeros((vocab_Size, 50))
for word, i in word_Index.items():
    embedding_Vector = embedding_index.get(word)
    if embedding_Vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_Matrix[i] = embedding_Vector

print (embedding_Matrix.shape)

(104728, 50)


In [139]:
embedding_layer = Embedding(vocab_Size,Embedding_dim,input_length=MAX_SEQUENCE_LENGTH,weights=[embedding_Matrix],trainable=False)
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(64, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(4)(x)
x = Conv1D(64, 5, activation='relu')(x)
x = MaxPooling1D(4)(x)
x = Conv1D(64, 5, activation='relu')(x)
x = MaxPooling1D(4)(x)  # global max pooling
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)

In [140]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [142]:
model.fit(x_train, y_train, batch_size = 50, epochs=25)

Epoch 1/25
15998/15998 [==============================] - 72s 4ms/step - loss: 2.3270 - accuracy: 0.2300
Epoch 2/25
15998/15998 [==============================] - 90s 6ms/step - loss: 1.7180 - accuracy: 0.4107
Epoch 3/25
15998/15998 [==============================] - 89s 6ms/step - loss: 1.4676 - accuracy: 0.4946
Epoch 4/25
15998/15998 [==============================] - 83s 5ms/step - loss: 1.3062 - accuracy: 0.5566
Epoch 5/25
15998/15998 [==============================] - 83s 5ms/step - loss: 1.1755 - accuracy: 0.5948
Epoch 6/25
15998/15998 [==============================] - 74s 5ms/step - loss: 1.0771 - accuracy: 0.6290
Epoch 7/25
15998/15998 [==============================] - 85s 5ms/step - loss: 0.9894 - accuracy: 0.6580
Epoch 8/25
15998/15998 [==============================] - 79s 5ms/step - loss: 0.9062 - accuracy: 0.6855
Epoch 9/25
15998/15998 [==============================] - 83s 5ms/step - loss: 0.8328 - accuracy: 0.7123
Epoch 10/25
15998/15998 [==============================

In [143]:
score = model.evaluate(x_test, y_test, batch_size=50)

print('Test Loss:', score[0])
print('Test Accuracy:', score[1])

4000/4000 [==============================] - 6s 1ms/step
Test Loss: 2.44290092587471
Test Accuracy: 0.5682500004768372


In [ ]:
Y_pred = model.predict(x_test)

In [ ]:
y_pred =[]
for i in Y_pred:
    y_pred.append(np.argmax(i))